In [1]:
import pandas as pd
import numpy as np
import operator
import ast
from torch.autograd import Variable
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import torch
import torch.nn as nn
import string
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence 
import spacy
import re

## Import Data

In [6]:
data = pd.read_csv('combined-newsqa-data-v1.csv')
print(data.columns)
print(type(data['is_question_bad'][0]))

# remove Q/A pairs that are invalid or missing
#data = data[(data.is_question_bad=='0.0') & (data.is_answer_absent=='0.0')]
data.head()

Index(['story_id', 'question', 'answer_char_ranges', 'is_answer_absent',
       'is_question_bad', 'validated_answers', 'story_text'],
      dtype='object')
<class 'str'>


,story_id,question,answer_char_ranges,is_answer_absent,is_question_bad,validated_answers,story_text
0,./cnn/stories/42d01e187213e86f5fe617fe32e716ff...,What was the amount of children murdered?,294:297|None|None,0.0,0.0,"{""none"": 1, ""294:297"": 2}","NEW DELHI, India (CNN) -- A high court in nort..."
1,./cnn/stories/c48228a52f26aca65c31fad273e66164...,Where was one employee killed?,34:60|1610:1618|34:60,0.0,0.0,NaN,(CNN) -- Fighting in the volatile Sudanese reg...
2,./cnn/stories/c65ed85800e4535f4bbbfa2c34d7d963...,who did say South Africa did not issue a visa ...,103:127|114:127|839:853,0.0,0.0,"{""839:853"": 1, ""103:127"": 2}",Johannesburg (CNN) -- Miffed by a visa delay t...
3,./cnn/stories/0cf66b646e9b32076513c050edf32a79...,How many years old was the businessman?,538:550|538:550,0.0,0.0,NaN,(CNN) -- England international footballer Ste...
4,./cnn/stories/13012604e3203c18df09289dfedd14cd...,What frightened the families?,690:742|688:791|630:646,0.0,0.0,"{""688:791"": 2, ""690:742"": 1}","BAGHDAD, Iraq (CNN) -- At least 6,000 Christi..."


In [3]:
print(data['is_question_bad'][0])

0.0


In [4]:
# 119,633 Q/A's , 12088 articles
print(len(data))
print(len(data['story_text'].unique()))


119633
12088


In [5]:
first_doc = data['story_text'][0]
first_doc

'NEW DELHI, India (CNN) -- A high court in northern India on Friday acquitted a wealthy businessman facing the death sentence for the killing of a teen in a case dubbed "the house of horrors."\n\n\n\nMoninder Singh Pandher was sentenced to death by a lower court in February.\n\n\n\nThe teen was one of 19 victims -- children and young women -- in one of the most gruesome serial killings in India in recent years.\n\n\n\nThe Allahabad high court has acquitted Moninder Singh Pandher, his lawyer Sikandar B. Kochar told CNN.\n\n\n\nPandher and his domestic employee Surinder Koli were sentenced to death in February by a lower court for the rape and murder of the 14-year-old.\n\n\n\nThe high court upheld Koli\'s death sentence, Kochar said.\n\n\n\nThe two were arrested two years ago after body parts packed in plastic bags were found near their home in Noida, a New Delhi suburb. Their home was later dubbed a "house of horrors" by the Indian media.\n\n\n\nPandher was not named a main suspect by 

In [6]:
first_doc[294:297]

'19 '

In [7]:
second_doc = data['story_text'][2]
second_doc

'Johannesburg (CNN) -- Miffed by a visa delay that led the Dalai Lama to cancel a trip to South Africa, Archbishop Desmond Tutu lashed out at his government Tuesday, saying it had acted worse than apartheid regimes and had forgotten all that the nation stood for.\n\n\n\n\n\n"When we used to apply for passports under the apartheid government, we never knew until the last moment what their decision was," Tutu said at a news conference. "Our government is worse than the apartheid government because at least you were expecting it from the apartheid government.\n\n\n\n\n\n"I have to say that I can\'t believe this. I really can\'t believe this," Tutu said. "You have to wake me up and tell me this is actually happening here."\n\n\n\n\n\nThe Dalai Lama scrapped his planned trip to South Africa this week after the nation failed to issue him a visa in time, his spokesman said.\n\n\n\n\n\nVisa applications for him and his entourage were submitted to the South African High Commission in New Delhi,

In [8]:
print(second_doc[103:127])
print(second_doc[114:127])
print(second_doc[839:853])

Archbishop Desmond Tutu 
Desmond Tutu 
his spokesman 


Answers given by different human reviewers are in the answer_char_ranges column and the validated_answers column. These values are string index ranges within the document that represent the answer. For each row, we need to determine which character range is the best answer to use and extract it. There are a couple ways we could approach this problem, the first is to have a model that takes the question(str), the story(str), spits out an answer(str).

or we could have it spit back out the string indexes themselves. 

In [11]:
# Remove uneeded columns
df = data[['question','validated_answers','story_text']]
df = df.dropna()
df = df.iloc[0:10000] # reduce size for development purposes

df.head()

,question,validated_answers,story_text
0,What was the amount of children murdered?,"{""none"": 1, ""294:297"": 2}","NEW DELHI, India (CNN) -- A high court in nort..."
2,who did say South Africa did not issue a visa ...,"{""839:853"": 1, ""103:127"": 2}",Johannesburg (CNN) -- Miffed by a visa delay t...
4,What frightened the families?,"{""688:791"": 2, ""690:742"": 1}","BAGHDAD, Iraq (CNN) -- At least 6,000 Christi..."
6,Who is hiring?,"{""301:324"": 2}",CNN affiliates report on where job seekers are...
8,Iran criticizes who?,"{""63:97"": 2}","TEHRAN, Iran (CNN) -- Iran's parliament speake..."


In [12]:
# loops through the rows and prints the question along with the first answer given
start_truth = []
end_truth = []
for i, row in df.iterrows():
    try:
        answers = ast.literal_eval(row['validated_answers'])
        sorted_ans = sorted(answers.items(), key=operator.itemgetter(1), reverse=True)
        #print(sorted_ans)
        start, end = sorted_ans[0][0].split(':')
        start_truth.append(int(start))
        end_truth.append(int(end))
    except ValueError:
        start_truth.append(np.nan)
        end_truth.append(np.nan)
        pass
df['start_truth'] = start_truth
df['end_truth'] = end_truth
df = df.dropna()
print(len(df))
df.head()

8318


,question,validated_answers,story_text,start_truth,end_truth
0,What was the amount of children murdered?,"{""none"": 1, ""294:297"": 2}","NEW DELHI, India (CNN) -- A high court in nort...",294.0,297.0
2,who did say South Africa did not issue a visa ...,"{""839:853"": 1, ""103:127"": 2}",Johannesburg (CNN) -- Miffed by a visa delay t...,103.0,127.0
4,What frightened the families?,"{""688:791"": 2, ""690:742"": 1}","BAGHDAD, Iraq (CNN) -- At least 6,000 Christi...",688.0,791.0
6,Who is hiring?,"{""301:324"": 2}",CNN affiliates report on where job seekers are...,301.0,324.0
8,Iran criticizes who?,"{""63:97"": 2}","TEHRAN, Iran (CNN) -- Iran's parliament speake...",63.0,97.0


In [13]:
final = df.drop(columns=['validated_answers'])
final['ss'] = np.zeros(len(final))
final.head()

,question,story_text,start_truth,end_truth,ss
0,What was the amount of children murdered?,"NEW DELHI, India (CNN) -- A high court in nort...",294.0,297.0,0.0
2,who did say South Africa did not issue a visa ...,Johannesburg (CNN) -- Miffed by a visa delay t...,103.0,127.0,0.0
4,What frightened the families?,"BAGHDAD, Iraq (CNN) -- At least 6,000 Christi...",688.0,791.0,0.0
6,Who is hiring?,CNN affiliates report on where job seekers are...,301.0,324.0,0.0
8,Iran criticizes who?,"TEHRAN, Iran (CNN) -- Iran's parliament speake...",63.0,97.0,0.0


## Char-level embeddings

In [14]:
full_text = ''
m = 0
len_nums = []

for col in ['question','story_text']:
    for text in final[col]:
        m = max(m, len(text))
        full_text += text.lower()
    
    len_nums.append(m)
    m = 0
    
# get the set of all characters
characters = tuple(set(full_text))
characters[:5]

('”', '6', 'á', 'd', 'ä')

In [15]:
# use enumeration to give the characters integer values
int2char = dict(enumerate(characters))

# create the look up dictionary from characters to the assigned integers
char2int = {char: index for index, char in int2char.items()}
print(char2int)


{'”': 0, '6': 1, 'á': 2, 'd': 3, 'ä': 4, 'é': 5, 'è': 6, '4': 7, 'r': 8, '1': 9, '_': 10, 'l': 11, 'ÿ': 12, '!': 13, '—': 14, 'z': 15, '[': 16, 'a': 17, ']': 18, 'w': 19, 'û': 20, 'í': 21, 'n': 22, 'ü': 23, ' ': 24, 'ţ': 25, 'c': 26, 'ò': 27, 'b': 28, '5': 29, '•': 30, '¯': 31, 'u': 32, '2': 33, '3': 34, '\xa0': 35, '(': 36, '\xad': 37, '9': 38, 'y': 39, 'ï': 40, 'ë': 41, 'ã': 42, 'q': 43, 'm': 44, 'ö': 45, '½': 46, '€': 47, 'f': 48, '×': 49, '8': 50, '*': 51, 'ñ': 52, '@': 53, '¡': 54, 'h': 55, 'k': 56, 'à': 57, '>': 58, ';': 59, '\\': 60, '|': 61, '0': 62, '¥': 63, '¢': 64, '+': 65, '¬': 66, '¹': 67, '\n': 68, '#': 69, '©': 70, 'ú': 71, '‚': 72, 's': 73, '"': 74, '=': 75, 'þ': 76, '¨': 77, '£': 78, '`': 79, 'â': 80, 'j': 81, 'ê': 82, '\u202a': 83, 'i': 84, ')': 85, ':': 86, 'ó': 87, '?': 88, '/': 89, '»': 90, 'ø': 91, 'o': 92, '7': 93, '§': 94, ',': 95, 'ş': 96, '·': 97, '-': 98, "'": 99, '¾': 100, '´': 101, 'ù': 102, 'v': 103, 'x': 104, 'e': 105, 'p': 106, 'ç': 107, 't': 108, '&': 1

In [16]:
q_max, story_max = len_nums
print(q_max, story_max)

112 11349


In [17]:
def find_s(seq):
    for i, char in enumerate(seq):
        if char == 0:
            continue
        else: return i

In [18]:
padding_start = False

for i, row in final.iterrows():
    for col, N in zip(['question','story_text'], [q_max, story_max]):
        enc = np.zeros(N, dtype=np.int32)
        
        row[col] = np.array([char2int[char] for char in row[col].lower()])
        l = min(N, len(row[col]))
        
        if padding_start:
            enc[:l] = row[col][:l]
        else:
            enc[N-l:] = row[col][:l]
        
        final[col][i] = enc
        final['ss'][i] = N-l

final.head()

/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


,question,story_text,start_truth,end_truth,ss
0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",294.0,297.0,10114.0
2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",103.0,127.0,7243.0
4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",688.0,791.0,9077.0
6,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",301.0,324.0,5733.0
8,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",63.0,97.0,8539.0


In [19]:
print(final.iloc[1,1])
print(find_s(final.iloc[1,1]))
# find_s([0,0,0,0,0,0,1,2,4])

[  0   0   0 ...   8 108 110]
7243


## Train/Test Split

In [20]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(final[['question', 'story_text', 'ss']], 
                                                    final[['start_truth','end_truth']], 
                                                    test_size=0.3, 
                                                    random_state=42)

In [21]:
print(X_train.index)
X_train.head()

Int64Index([ 2620,  2165,  5896, 11169,  4408,  7251, 13871, 15569,  4459,
            16101,
            ...
             4609, 15400, 12242,  1326, 17310, 15819, 14346, 14897,  2295,
            20192],
           dtype='int64', length=5822)


,question,story_text,ss
2620,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",6931.0
2165,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",9698.0
5896,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",8949.0
11169,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",9016.0
4408,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",9033.0


In [22]:
min(y_train.end_truth)

6.0

In [23]:
y_train.head()

,start_truth,end_truth
2620,172.0,194.0
2165,339.0,357.0
5896,171.0,176.0
11169,120.0,130.0
4408,171.0,184.0


## Dataset

In [24]:
class NewsQADataset(Dataset):
    def __init__(self, X, y, N=400, padding_start=False):
        self.start = y['start_truth'].values
        self.end = y['end_truth'].values
        self.question = X['question'].values
        self.story_text = X['story_text'].values
        self.ss = X['ss'].values
        
    def __len__(self):
        return len(self.question)
    
    def __getitem__(self, idx):
        #print(self.question)
        #qs = np.array([find_s(seq) for seq in self.question])
        # ss = np.array([find_s(seq) for seq in self.story_text])
        return self.question[idx], self.story_text[idx], self.start[idx] + self.ss[idx], self.end[idx] + self.ss[idx]

In [25]:
train_ds = NewsQADataset(X_train, y_train)
test_ds = NewsQADataset(X_test, y_test)

In [26]:
train_ds[0]

(array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,  19,  55,  92,  24,  11, 105,  48, 108,
         24,  44, 105, 104,  84,  26,  92,  99,  73,  24, 113,  92, 103,
        105,   8,  22,  44, 105,  22, 108,  88], dtype=int32),
 array([  0,   0,   0, ...,  84,   3, 110], dtype=int32),
 7103.0,
 7125.0)

In [27]:
batch_size = 100
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
test_dl = DataLoader(test_ds, batch_size=batch_size)

In [28]:
q, s, start, end = next(iter(train_dl))
print(q.shape, s.shape, start.shape, end.shape)
q, s, start, end

torch.Size([100, 112]) torch.Size([100, 11349]) torch.Size([100]) torch.Size([100])


(tensor([[  0,   0,   0,  ...,  24, 108,  92],
         [  0,   0,   0,  ...,  24,  17,  88],
         [  0,   0,   0,  ...,  17,   8,  88],
         ...,
         [  0,   0,   0,  ..., 105,   3,  88],
         [  0,   0,   0,  ..., 108,  73,  88],
         [  0,   0,   0,  ...,   8, 105,  88]], dtype=torch.int32),
 tensor([[  0,   0,   0,  ...,   1,  62, 110],
         [  0,   0,   0,  ...,  11,  73, 110],
         [  0,   0,   0,  ...,   8,  39, 110],
         ...,
         [  0,   0,   0,  ...,   8, 108, 110],
         [  0,   0,   0,  ..., 105, 110,  74],
         [  0,   0,   0,  ...,   8, 108, 110]], dtype=torch.int32),
 tensor([ 5302.,  8527.,  6690.,  9734., 10574.,  7930.,  7244.,  6546., 10956.,
          9978.,  4971.,  9016.,  9632.,  8715.,  7250.,  7938.,  9447., 11214.,
         11288., 10234., 10757., 10161.,  8053.,  9583.,  8567.,  7859.,  9464.,
         11003.,  9468.,  7713.,  9911.,  7393.,  8659.,  7364., 10311.,  7667.,
          6907., 10718., 10955.,  7604.,  

## Modeling

In [4]:
class GRUModel(torch.nn.Module) :
    def __init__(self, vocab_size, embedding_dim, hidden_dim, q_max, story_max):
        super(GRUModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.dropout = nn.Dropout(0.5)
        self.gru = nn.GRU(embedding_dim, hidden_dim, batch_first=True)
        self.linear2 = nn.Linear(200, story_max)
        self.linear_start = nn.Linear(story_max, story_max+1)
        self.linear_end = nn.Linear(story_max, story_max+1)
        
    def forward(self, x1, x2):
        
        x1 = self.embeddings(x1)
        x1 = self.dropout(x1)
        out_pack1, ht1= self.gru(x1)
        
        x2 = self.embeddings(x2)
        x2 = self.dropout(x2)
        out_pack2, ht2= self.gru(x2)
        
        
        x = torch.cat((ht1[-1], ht2[-1]),1)
        
        x = F.relu(self.linear2(x))
        start = self.linear_start(x)
        end = self.linear_end(x)
 
        return start, end

NameError: name 'torch' is not defined

I've taken the functions from her last name classification model because that uses character-level embeddings as well. I'm using a naive version that just concats the question with the story and uses that as input into the RNN. I used that with regressing 2 outputs (start, end) and I got it to train but not sure how well it's actually doing. I think we correct way to do this is to treat it as a classification problem where each index of the story is a categroy and we use those predictions

In [9]:
def get_optimizer(model, lr = 0.1, wd = 0.0001):
    parameters = filter(lambda p: p.requires_grad, model.parameters())
    optim = torch.optim.Adam(parameters, lr=lr, weight_decay=wd)
    return optim

def vectorize(labels, size=7361):
    result = []
    for s in labels:
        #print(s)
        row = np.zeros(size)
        row[int(s)] = 1
        result.append(row)
        
    return result

def custom_metric(start_real, end_real, start_pred, end_pred):
    # len(intersection) / len(union)
    vals = []
    for s1, e1, s2, e2 in zip(start_real, end_real, start_pred, end_pred):
        real_range = set(range(s1, e1+1, 1))
        pred_range = set(range(s2, e2+1, 1))
        intersection = real_range.intersection(pred_range)
        union = real_range.union(pred_range)
        vals.append(len(intersection)/len(union))
        
    return np.mean(vals)
        

custom_metric([1,2,3],[2,3,4],[1,2,3],[3,4,5])

0.6666666666666666

In [34]:
def train_epocs(model, epochs=5, lr=0.01):
    parameters = filter(lambda p: p.requires_grad, model.parameters())
    optimizer = torch.optim.Adam(parameters, lr=lr)
    for i in range(epochs):
        model.train()
        sum_loss = 0.0
        total = 0
        for q, s, start_, end_ in train_dl:
            #start_vec = vectorize(start_)
            #end_vec = vectorize(end_)
            q = q.long()
            s = s.long()
            start_ = start_.long()
            end_ = end_.long()

            start, end = model(q, s)
            loss_start = F.cross_entropy(start, start_)

            loss_end = F.cross_entropy(end, end_)
            loss = loss_start + loss_end
            
        
            loss.backward()
            optimizer.step()
            sum_loss += loss.item()*end_.shape[0]
            total += end_.shape[0]
        val_loss, val_score = val_metrics(model, test_dl)
        #if i % 5 == 1:
        print("train loss %.3f val loss %.3f and val score %.3f" % (sum_loss/total, val_loss, val_score))
            
def val_metrics(model, valid_dl):
    model.eval()
    correct = 0
    total = 0
    sum_loss = 0.0
    scores = []
    for q, s, start_, end_ in valid_dl:
        q = q.long()
        s = s.long()
        start_ = start_.long()
        end_ = end_.long()

        start, end = model(q, s)
        #print(start, end)
        #print(start.shape, end.shape)
        
        loss_start = F.cross_entropy(start, start_)
        #print(loss_start)
        
        loss_end = F.cross_entropy(end, end_)
        #print(loss_end)
        
        loss = loss_start + loss_end
        #print(loss)
        #
        start_i = start.argmax(1)
        end_i = end.argmax(1)
        #for st, en in zip(start_i, end_i):
        
        score = custom_metric(start_, end_, start_i, end_i)
        scores.append(score)
        
        
        total += end_.shape[0]
        sum_loss += loss.item()*end_.shape[0]
    print(f'Predicted: {start_i}, Real: {start_}')
    #print(f'Real: {start_}, {end_}')
    return sum_loss/total, np.mean(scores)

In [35]:
vocab_size = len(char2int)
hidden_size = 100
n_classes = 2
emb_size = 10
model = GRUModel(vocab_size, emb_size, hidden_size, q_max, story_max)

# val_metrics(model, test_dl)

In [36]:
train_epocs(model)

Predicted: tensor([ 9787,  9787,  9787,  9787,  9787,  9787,  9787,  9787,  9787,  9787,
         9787,  9787,  9787,  9787,  9787,  9787,  9787,  9787,  9787,  9787,
         9787,  9787,  9787,  9787,  9787,  9787,  9787,  9787,  9787,  9787,
         9787,  9787,  9787,  9787,  9787,  9787,  9787,  9787,  9787,  9787,
         9787,  9787,  9787,  9787,  9787,  9787,  9787,  9787,  9787,  9787,
         9787,  9787,  9787,  8983,  9787,  9787,  9787,  9787,  9787,  9787,
         9787,  9787,  9787,  9787,  9787,  9787,  9787,  9787,  9787,  9787,
         9787,  9787,  9787,  9787,  9787,  9787,  9787,  9787,  9787,  9787,
         9787, 10040,  9787,  9787,  9787,  9787,  9787,  9787,  9787,  9787,
         9787,  9787,  9787,  9787,  9787,  9787]), Real: tensor([11291,  9028, 10073,  9535, 10972,  9580,  7871,  9072,  9756,  8948,
         9542,  9075,  9727, 10498,  8772,  8125,  8138,  9561, 10093,  8738,
         7210,  8869,  9947, 10124,  7153, 10803,  5007,  6941,  9800,  8

In [3]:
list(range(4,10,1))

[4, 5, 6, 7, 8, 9]